In [38]:
import numpy as np

def make_contreras_snail(label="snail", 
                         b = .1, d = 4, z = 1, a = 5, phi = 0, psi = 0, 
                         c_depth=0.1, c_n = 10, n_depth = 0, n = 0, 
                         h_0 = 1, eps = 0.5,
                         time = 20, n_points_time = 1000, 
                         n_points_aperture=15):

    # snail axes are [XYZ][Aperture Angle Theta][Time]
    gamma: np.array = np.zeros((3, n_points_time, n_points_aperture))

    # vector of points in time reshaped for broadcasting
    t: np.array = np.linspace(0, time, n_points_time).reshape((n_points_time, 1))
    theta: np.array = np.linspace(0, 2*np.pi, n_points_aperture).reshape((1, n_points_aperture))

    # precalculating some repeated operations for efficiency
    sin_t = np.sin(t)
    cos_t = np.cos(t)
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    bsq = b**2
    dsq = d**2

    radial_ribbing = (1 + n_depth + n_depth*np.sin(n*theta))
    spiral_ribbing = (1 + c_depth + c_depth*np.sin(c_n*t))

    # shape = (3, n_points_time, 1) -> [xyz][ti)me][constant with respect to the angle theta along the aperture]
    gamma = np.exp(b*t)*np.array([
        d*sin_t, d*cos_t, 
        np.full(n_points_time, z).reshape(n_points_time, 1)
    ])

    # Defining the normal and binormal vectors along the frennet frame for all time points and angles about the aperture
    # 3 x n_points_time x 1 for both
    N: np.array = np.array([
        b*cos_t - sin_t,
        -b*sin_t - cos_t,
        np.zeros((n_points_time, 1))
    ])/np.sqrt(b**2 + 1)

    B: np.array = np.array([
        b*z*(b*sin_t + cos_t),
        b*z*(b*cos_t - sin_t),
        np.full((n_points_time, 1), d*(bsq + 1))
    ])/np.sqrt(
        (bsq + 1)*((bsq + 1)*dsq + bsq*(z**2))
    )

    # Define the rotation matrix for the aperture given that psi 
    # is the rotation angle about the B axis in the local frennet frame
    R: np.array = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    
    # Define the unrotated and unscaled generating curve
    # Shape = (1, 1, n_points_aperture) | constant for [xyz][time], [changes for the angle]
    GC: np.array = (a*sin_theta*np.sin(phi) - cos_theta*np.cos(phi)).reshape(1, 1, n_points_aperture)*B + (a*sin_theta*np.cos(phi) + cos_theta*np.sin(phi)).reshape(1, 1, n_points_aperture)*N
    GC_outer: np.array = radial_ribbing*GC 

    # einsum lets me broadcast the rotation matrix multiplication over the time points and the aperture angles
    rGC_outer = np.einsum('ij,jkl->ikl', R, GC_outer)
    rGC = np.einsum('ij,jkl->ikl', R, GC)
    timescale = (np.exp(b*t) - (1 / (t + 1)))
    #adjusting the wave height from 1 to (1+c_depth) so the percentage is always > 100%
    outer_timescale = spiral_ribbing*timescale
    # eps is a modified version of the shell width parameter described by Okabe 2017
    inner_timescale = timescale - ((timescale**eps)*h_0)

    # We tranpose the angle and time axes so the final shape is [XYZ][Aperture][Time], which is easier to index for mesh
    outer_mesh = np.transpose(gamma + (outer_timescale * rGC_outer), (0, 2, 1))
    inner_mesh = np.transpose(gamma + (inner_timescale * rGC), (0, 2, 1))

    # have inner and outer vertices into one array
    outer_mesh_vertices = outer_mesh.reshape(3, n_points_aperture*n_points_time)
    inner_mesh_vertices = inner_mesh.reshape(3, n_points_aperture*n_points_time)

    # Reshape the inner and outer meshes so their shapes are compatable with Blender
    return {"label": label,
            "outer_mesh": outer_mesh,
            "outer_mesh_vertices": outer_mesh_vertices,
            "inner_mesh_vertices": inner_mesh_vertices
            }

n_points_time = 15
n_points_aperture=10

snail = make_contreras_snail(z = 1.3, a = 1, d=1, phi=0, psi=0,
                             b=.15,
                             n_depth=0, n=0, 
                             c_n=0, c_depth=0,  
                             time=200, n_points_time=n_points_time, 
                             n_points_aperture=n_points_aperture, 
                             h_0 = 40, eps=.8)

shell = snail['outer_mesh_vertices']

In [44]:
indices = np.arange(n_points_aperture*n_points_time).reshape(n_points_time, n_points_aperture)

In [63]:
indexes = np.arange(n_points_aperture*n_points_time)
inner_indexes = indexes[(indexes + 1) % n_points_aperture != 0]
outer_indexes = np.setdiff1d(indexes, inner_indexes)

expanded_inner_indexes = np.stack([
    inner_indexes,
    inner_indexes + 1,
    inner_indexes + n_points_aperture,
    inner_indexes + n_points_aperture + 1
], axis=1)

expanded_outer_indexes = np.stack([
    outer_indexes,
    (outer_indexes - n_points_aperture) + 1,
    outer_indexes + n_points_aperture,
    outer_indexes + 1
], axis=1)

expanded_indexes = np.concatenate([expanded_inner_indexes, expanded_outer_indexes], axis=0)
print(expanded_indexes)

[[  0   1  10  11]
 [  1   2  11  12]
 [  2   3  12  13]
 [  3   4  13  14]
 [  4   5  14  15]
 [  5   6  15  16]
 [  6   7  16  17]
 [  7   8  17  18]
 [  8   9  18  19]
 [ 10  11  20  21]
 [ 11  12  21  22]
 [ 12  13  22  23]
 [ 13  14  23  24]
 [ 14  15  24  25]
 [ 15  16  25  26]
 [ 16  17  26  27]
 [ 17  18  27  28]
 [ 18  19  28  29]
 [ 20  21  30  31]
 [ 21  22  31  32]
 [ 22  23  32  33]
 [ 23  24  33  34]
 [ 24  25  34  35]
 [ 25  26  35  36]
 [ 26  27  36  37]
 [ 27  28  37  38]
 [ 28  29  38  39]
 [ 30  31  40  41]
 [ 31  32  41  42]
 [ 32  33  42  43]
 [ 33  34  43  44]
 [ 34  35  44  45]
 [ 35  36  45  46]
 [ 36  37  46  47]
 [ 37  38  47  48]
 [ 38  39  48  49]
 [ 40  41  50  51]
 [ 41  42  51  52]
 [ 42  43  52  53]
 [ 43  44  53  54]
 [ 44  45  54  55]
 [ 45  46  55  56]
 [ 46  47  56  57]
 [ 47  48  57  58]
 [ 48  49  58  59]
 [ 50  51  60  61]
 [ 51  52  61  62]
 [ 52  53  62  63]
 [ 53  54  63  64]
 [ 54  55  64  65]
 [ 55  56  65  66]
 [ 56  57  66  67]
 [ 57  58  6